# NHL predictions project - Data Interpretation

This is project for the subject Data Processing in Python - JEM207. That was created by Pavlína Křenková and Vojtěch Vaverka.

The code we wrote scrapes data from the website https://nhl.cz/. After the data is scraped, user of the code needs to fill in names of two teams into the variables team_1 and team_2. Once those two variables are set the code creates a Logistic regression that predicts which team is going to win. Druing our testing of the code we have reached accuracy above 50%. That means that the model is better than randomly guessing. Yet it should not be used to place bets.

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import seaborn as sns
from statistics import mean
from ipywidgets import Dropdown

We load the necessary information about the teams, which we will later use for file loading and analysis

In [2]:
teams_info = pd.read_csv('teams_info/teams_info.csv')
#teams_info.head()

In [3]:
#We will be analysing the 5 most recent seasons
seasons_used = ['2017', '2018', '2019', '2020','2021'] #2017 is the 2017/2018 season, etc.

In [4]:
distinct_teams = teams_info['team'].tolist()

In [5]:
teams_dict = pd.Series(teams_info.team_short.values,index=teams_info.team).to_dict()
teams_dict

{'Anaheim Ducks': 'ANA',
 'Arizona Coyotes': 'ARI',
 'Boston Bruins': 'BOS',
 'Buffalo Sabres': 'BUF',
 'Calgary Flames': 'CGY',
 'Carolina Hurricanes': 'CAR',
 'Chicago Blackhawks': 'CHI',
 'Colorado Avalanche': 'COL',
 'Columbus Blue Jackets': 'CBJ',
 'Dallas Stars': 'DAL',
 'Detroit Red Wings': 'DET',
 'Edmonton Oilers': 'EDM',
 'Florida Panthers': 'FLA',
 'Los Angeles Kings': 'LAK',
 'Minnesota Wild': 'MIN',
 'Montreal Canadiens': 'MTL',
 'Nashville Predators': 'NSH',
 'New Jersey Devils': 'NJD',
 'New York Islanders': 'NYI',
 'New York Rangers': 'NYR',
 'Ottawa Senators': 'OTT',
 'Philadelphia Flyers': 'PHI',
 'Pittsburgh Penguins': 'PIT',
 'San Jose Sharks': 'SJS',
 'Seattle Kraken': 'SEA',
 'St. Louis Blues': 'STL',
 'Tampa Bay Lightning': 'TBL',
 'Toronto Maple Leafs': 'TOR',
 'Vancouver Canucks': 'VAN',
 'Vegas Golden Knights': 'VGK',
 'Washington Capitals': 'WSH',
 'Winnipeg Jets': 'WPG'}

## Choosing two teams and building the model

In [25]:
teams_info = pd.read_csv('teams_info/teams_info.csv')
option_list = teams_info['team'].tolist()
option_list.insert(0, '')
team_1_drop = ''
team_2_drop = ''
drop_down_team_1 = Dropdown(options=option_list,description='Choose the home team (Team 1):',disabled=False, 
                     display='flex',
                     flex_flow='column', align_items='stretch', style= {'description_width': 'initial'},
                     layout={'width': 'max-content'})
drop_down_team_2 = Dropdown(options=option_list,description='Choose the away team (Team 2):',disabled=False, 
                     display='flex',
                     flex_flow='column', align_items='stretch', style= {'description_width': 'initial'},
                     layout={'width': 'max-content'})
def dropdown_handler_team_1(choice):
    global team_1_drop
    print("Team 1:", choice.new)
    team_1_drop = choice.new
def dropdown_handler_team_2(choice):
    global team_2_drop
    print("Team 2:", choice.new)
    team_2_drop = choice.new
drop_down_team_1.observe(dropdown_handler_team_1, names='value')
drop_down_team_2.observe(dropdown_handler_team_2, names='value')
display(drop_down_team_1, drop_down_team_2)

Dropdown(description='Choose the home team (Team 1):', layout=Layout(width='max-content'), options=('', 'Anahe…

Dropdown(description='Choose the away team (Team 2):', layout=Layout(width='max-content'), options=('', 'Anahe…

Team 1: Boston Bruins
Team 2: New York Rangers
Team 1: Columbus Blue Jackets
Team 2: Vancouver Canucks


In [32]:
if team_1_drop == "" or team_2_drop == "":
    print("You have not chosen both teams yet, please return to previous cell and do so.")
elif team_1_drop == team_2_drop:
    print("You picked the same team twice, please return to previous cell and change at least one team.")
else:
    print("The outcome of the match", team_1_drop, "vs ", team_2_drop, "will be predicted.")
    team_1 = team_1_drop
    team_2 = team_2_drop

The outcome of the match Columbus Blue Jackets vs  Vancouver Canucks will be predicted.


In [27]:
team_1_short = teams_dict[team_1]
team_2_short = teams_dict[team_2]

In [16]:
matches_team_1 = pd.read_csv(f'teams_final/{team_1_short}.csv')
matches_team_1.head()

,game_id,TOI_result,TOI_home,TOI,other_team,TOI_points_tot,TOI_goals_scored_tot,TOI_goals_rec_tot,other_team_points_tot,other_team_goals_scored_tot,other_team_goals_rec_tot
0,/zapas/2526364,0.0,1.0,Buffalo Sabres,Columbus Blue Jackets,18.0,46.0,71.0,31.0,59.0,51.0
1,/zapas/2526373,0.0,1.0,Buffalo Sabres,Minnesota Wild,17.0,46.0,71.0,30.0,59.0,55.0
2,/zapas/2526393,1.0,1.0,Buffalo Sabres,Edmonton Oilers,17.0,47.0,70.0,20.0,54.0,71.0
3,/zapas/2526403,0.0,0.0,Buffalo Sabres,Montreal Canadiens,20.0,48.0,65.0,22.0,50.0,70.0
4,/zapas/2526421,0.0,1.0,Buffalo Sabres,Tampa Bay Lightning,20.0,46.0,65.0,42.0,73.0,51.0


In [17]:
#dropping unnecessary columns
matches_team_1 = matches_team_1.drop(columns=['game_id', 'TOI','other_team'])

#### Defining the independent and dependent variables

In [18]:
#independent
x = matches_team_1.iloc[:, 1:].values
#dependent - the result
y = matches_team_1.iloc[:, 0].values

#### Splitting into training and test set

In [19]:
matches_team_1.iloc[:, 1:]

,TOI_home,TOI_points_tot,TOI_goals_scored_tot,TOI_goals_rec_tot,other_team_points_tot,other_team_goals_scored_tot,other_team_goals_rec_tot
0,1.0,18.0,46.0,71.0,31.0,59.0,51.0
1,1.0,17.0,46.0,71.0,30.0,59.0,55.0
2,1.0,17.0,47.0,70.0,20.0,54.0,71.0
3,0.0,20.0,48.0,65.0,22.0,50.0,70.0
4,1.0,20.0,46.0,65.0,42.0,73.0,51.0
...,...,...,...,...,...,...,...
345,0.0,30.0,61.0,72.0,18.0,54.0,81.0
346,0.0,33.0,65.0,69.0,21.0,63.0,78.0
347,1.0,33.0,67.0,70.0,33.0,58.0,58.0
348,0.0,33.0,67.0,71.0,35.0,66.0,54.0


In [20]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, random_state = 0)

In [21]:
X_test[:5]

array([[ 0., 17., 41., 66., 28., 53., 61.],
       [ 1., 27., 55., 65., 36., 56., 48.],
       [ 1., 24., 61., 72., 20., 56., 70.],
       [ 1., 26., 52., 56., 41., 71., 54.],
       [ 0., 15., 52., 82., 39., 84., 56.]])

In [22]:
#rescaling everything except the "home" dummy ... and then adding the dummy back
scaler = StandardScaler()
X_train_stand = scaler.fit_transform(X_train[:,1:])
X_test_stand = scaler.transform(X_test[:,1:])
X_train_stand = np.insert(X_train_stand,0,X_train[:,0],axis=1)
X_test_stand = np.insert(X_test_stand,0,X_test[:,0],axis=1)

#### The model and predictions

In [ ]:
reg = LogisticRegression(max_iter = 400)
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)

In [ ]:
#how well does the model perform?
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)

In [ ]:
print("Accuracy :  \n", accuracy_score(y_test, y_pred))

In [ ]:
matches_team_2 = pd.read_csv(f'teams_final/{team_2_short}.csv')
matches_team_2.head()

### the final prediction

In [ ]:
matches_team_1_pre = pd.read_csv(f'teams_pre-processed/{team_1_short}_pre.csv')
matches_team_2_pre = pd.read_csv(f'teams_pre-processed/{team_2_short}_pre.csv')

In [ ]:
pred_team_1 = matches_team_1_pre[['TOI_points', 'TOI_goals_scored', 'TOI_goals_rec']].tail(20).sum().values

In [ ]:
#je to v poradi points-goals scored-goals received
pred_team_1

In [ ]:
pred_team_2 = matches_team_2_pre[['TOI_points', 'TOI_goals_scored', 'TOI_goals_rec']].tail(20).sum().values
pred_team_2

In [ ]:
TOI_home = np.array([1])

In [ ]:
values_to_fit = np.concatenate([TOI_home, pred_team_1, pred_team_2]).reshape(1, -1)

In [ ]:
final_pred = reg.predict(values_to_fit)
final_pred

In [ ]:
if final_pred[0] == 1:
    print('The team', team_1, "is predicted to win this game.")
elif final_pred[0] == 0:
    print('The team', team_2, "is predicted to win this game.")
else:
    print('An error occured along the way. Please try again.')

## Accuraccy of models of individual teams

In [ ]:
#How accurate is the model for individual teams?
accuracies = []
for one_team in distinct_teams:
    one_team_short = teams_dict[one_team]
    matches_one_team = pd.read_csv(f'teams_final/{one_team_short}.csv')
    matches_one_team = matches_one_team.drop(columns=['game_id', 'TOI','other_team'])
    x = matches_one_team.iloc[:, 1:].values
    y = matches_one_team.iloc[:, 0].values
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, random_state = 0)

    scaler = StandardScaler()
    X_train_stand = scaler.fit_transform(X_train[:,1:])
    X_test_stand = scaler.transform(X_test[:,1:])
    X_train_stand = np.insert(X_train_stand,0,X_train[:,0],axis=1)
    X_test_stand = np.insert(X_test_stand,0,X_test[:,0],axis=1)
    reg = LogisticRegression(max_iter = 400)
    reg.fit(X_train, y_train)
    y_pred = reg.predict(X_test)
    accuracies.append(accuracy_score(y_test, y_pred))
    

print((accuracies))
print(sum(acc < 0.5 for acc in accuracies)) #4 teams out of 32 have lower then 0.5 accuracy

In [ ]:
mean(accuracies) #the model is 57% accurate, on average

### Calculating Home Advantage

In [ ]:
#ja moc nevim jestli tenhle vypocet je dobre
all_matches = pd.read_csv('all_matches.csv')
sum_wins_home = sum(all_matches["goals_home"] > all_matches["goals_away"])
sum_wins_away = sum(all_matches["goals_home"] < all_matches["goals_away"])
 
home_winrate = sum_wins_home/(sum_wins_home+sum_wins_away)
home_winrate

### calculating and visualising home advantage for all teams for all seasons together

In [ ]:
home_advantages = {}
for one_team in distinct_teams:
    one_team_short = teams_dict[one_team]
    matches_one_team = pd.read_csv(f'teams_final/{one_team_short}.csv')
    home_ad = (sum((matches_one_team['TOI_home'] == 1) & (matches_one_team['TOI_result'] == 1)))/sum(matches_one_team['TOI_home'] == 1)
    home_advantages[one_team] = home_ad
    
home_advantages

#add points total, so that we can see if it corresponds

### calculating and visualising home advantage for individual seasons

In [ ]:
home_advantage = pd.DataFrame(seasons_used, columns=['year'])
ha = []
for season in seasons_used:
    ha_one_season = sum(all_matches.loc[all_matches['season'] == int(season)]["home_wins"])/len(all_matches.loc[all_matches['season'] == int(season)]["home_wins"])
    ha.append(round(ha_one_season*100,3))
home_advantage["win_home_percentage"] = ha
home_advantage

In [ ]:
ha = home_advantage.plot(x = 'year', legend=False, kind = "bar")
plt.ylabel("Home Win %")
ha.bar_label(ha.containers[0], labels=home_advantage['win_home_percentage'])
plt.show()

### calculating home advantage for all teams and all seasons

In [ ]:
sum(all_matches["home_wins"])/len(all_matches["home_wins"])

### calculating and visualising home advantage for one specific team and all seasons

### After picking 2 teams both their final_dfs are found, saved and displayed

In [ ]:
matches_team_1s = pd.read_csv(f'teams_final/{team_1_short}.csv')
matches_between_them = matches_team_1s.merge(matches_team_2, left_on='game_id', right_on='game_id',suffixes=('', '2'))
matches_between_them.drop(columns=['TOI_result2', 'TOI_home2','TOI2','other_team2','TOI_points_tot2','TOI_goals_scored_tot2','TOI_goals_rec_tot2'], inplace=True)
pd.set_option("display.max_rows", None, "display.max_columns", None)
matches_between_them.head()

In [ ]:
pd.set_option("display.max_rows", 15, "display.max_columns", 15)

In [ ]:
matches_between_them_ids = matches_between_them['game_id']
matches_between_them_ids

### Graphs

#### We create data frames for graphs and we ad variable match_ordr that displays the order of matches of a given team

In [ ]:
matches_team_1_graphs = pd.read_csv(f'teams_pre-processed/{team_1_short}_pre.csv')
matches_team_2_graphs = pd.read_csv(f'teams_pre-processed/{team_2_short}_pre.csv')
matches_team_1_graphs['match_ordr'] = range(1, len(matches_team_1_graphs) + 1)
matches_team_2_graphs['match_ordr'] = range(1, len(matches_team_2_graphs) + 1)
matches_team_2_graphs.head()

### Merge both teams into one dataframe and create a graph of scored goals

In [ ]:
graphs = pd.concat([matches_team_1_graphs, matches_team_2_graphs], axis=0, ignore_index=False)
graphs['team'] = (len(matches_team_1_graphs)*(1,) + len(matches_team_2_graphs)*(2,))
graph_1 = sns.catplot(x='match_ordr', y='TOI_goals_scored', hue='team', kind='bar', data=graphs, height=10, aspect=10, palette=["#FA7F63", "#00FC35"])
graph_1.set(xticklabels=[])
plt.show()

#### Too many matches we need to limit the data to last 30 matches

In [ ]:
matches_team_1_graphs_30 = matches_team_1_graphs.tail(30)
matches_team_2_graphs_30 = matches_team_2_graphs.tail(30)
graphs_30_matches = pd.concat([matches_team_1_graphs_30, matches_team_2_graphs_30], axis=0, ignore_index=False)
graphs_30_matches['team'] = (len(matches_team_1_graphs_30)*(1,) + len(matches_team_2_graphs_30)*(2,))
graphs_30_matches
graph_2 = sns.catplot(x='match_ordr', y='TOI_goals_scored', hue='team', kind='bar', data=graphs_30_matches, height=5, aspect=3, palette=["#FA7F63", "#00FC35"])
graph_2.set(xticklabels=[])
plt.show()

### Recived Goals

In [ ]:
graph_3 = sns.catplot(x='match_ordr', y='TOI_goals_rec', hue='team', kind='bar', data=graphs_30_matches, height=5, aspect=3, palette=["#FA7F63", "#00FC35"])
graph_3.set(xticklabels=[])
plt.show()

### How teams did in last 30 matches

In [ ]:
ax = matches_team_1_graphs_30[['TOI_goals_scored','TOI_goals_rec','match_ordr']].plot(
    x='match_ordr',kind='bar',figsize=(15, 5),color=["#FA7F63", "#BF2806"],title='Team 1')
matches_team_1_graphs_30[['TOI_points','match_ordr']].plot(x='match_ordr',  linestyle='-', marker='*',figsize=(15, 5),use_index=False,
                                                   color=["#EEAC26"], lw=4, ax=ax)
ax.set(xticklabels=[])
plt.show

In [ ]:
ax = matches_team_2_graphs_30[['TOI_goals_scored','TOI_goals_rec','match_ordr']].plot(
    x='match_ordr',kind='bar',figsize=(15, 5),color=["#00FC35", "#42BB5C"],title='Team 2')
matches_team_2_graphs_30[['TOI_points','match_ordr']].plot(x='match_ordr',  linestyle='-', marker='*',figsize=(15, 5),use_index=False,
                                                   color=["#B6E968"], lw=4, ax=ax)
ax.set(xticklabels=[])
plt.show

### Comparison of important variables for our model

In [ ]:
x_coordinates = ['TOI_points', 'TOI_goals_scored', 'TOI_goals_rec']
X_axis = np.arange(len(x_coordinates))
plt.plot(x_coordinates, pred_team_1,'#FA7F63',label="Team 1") 
plt.plot(x_coordinates, pred_team_2,'#00FC35',label="Team 2") 
plt.legend(loc="upper left")
plt.show

### 2nd model based on matches between 2 teams

In [ ]:
#TOI = team 1 and should be home team to reflect home team advantage
second_model_mathes_filtered= matches_team_1_pre[matches_team_1_pre.other_team == team_2].copy()
second_model_mathes_filtered['match_ordr'] = range(1, len(second_model_mathes_filtered) + 1)
second_model_mathes_filtered

### 2nd model here we predict winner based on wins and loses in duels between teams

In [ ]:
model_2_data = sum(second_model_mathes_filtered.TOI_result)/len(second_model_mathes_filtered.TOI_result)
if model_2_data >= 0.5:
  print(team_1,"will win the match")
elif model_2_data < 0.5:
  print(team_2,"will win the match")
else:
  print("our model can not predict the winner as we do not have data  of", team_1,"facing ",team_2)

###  Another version of 2nd model here we predict winner based on points in duels between teams

In [ ]:
model_2_data_b = sum(second_model_mathes_filtered.TOI_points)/(len(second_model_mathes_filtered.TOI_result)*3)
if model_2_data_b >= 0.5:
      print(team_1,"will win the match")
elif model_2_data_b < 0.5:
      print(team_2,"will win the match")
else:
      print("our model can not predict the winner as we do not have data  of", team_1,"faceing ",team_2)

### Accuracy


In [ ]:
second_model_mathes_filtered_accuracy = second_model_mathes_filtered.iloc[:-1 , :] #we prepare a data set without the last match

In [ ]:
second_model_mathes_filtered_result = second_model_mathes_filtered.tail(1) #the result of last match we are checking against

### Another try at accuracy

In [ ]:
wins_before_this_match = []
points_before_this_match = []
matches_before_this_match = []
for x in second_model_mathes_filtered['match_ordr']:
    wins_before_this_match.append(second_model_mathes_filtered[(second_model_mathes_filtered['match_ordr'] < x)]['TOI_result'].sum())
    points_before_this_match.append(second_model_mathes_filtered[(second_model_mathes_filtered['match_ordr'] < x)]['TOI_points'].sum())
    matches_before_this_match.append(second_model_mathes_filtered[(second_model_mathes_filtered['match_ordr'] < x)]['TOI_points'].count())

second_model_mathes_filtered['wins_before_this_match'] = wins_before_this_match
second_model_mathes_filtered['points_before_this_match'] = points_before_this_match
second_model_mathes_filtered['matches_before_this_match'] = matches_before_this_match
second_model_mathes_filtered

In [ ]:
def predictions_score(row):
    if row['wins_before_this_match'] >= row['matches_before_this_match']/2:
        val = 1
    elif row['wins_before_this_match'] < row['matches_before_this_match']/2:
        val = 0
    return val    
second_model_mathes_filtered['Prediction_score'] = second_model_mathes_filtered.apply(predictions_score, axis=1)
def predictions_points(row):
    if row['wins_before_this_match'] >= row['matches_before_this_match']/2:
        val = 1
    elif row['wins_before_this_match'] < row['matches_before_this_match']/2:
        val = 0
    return val    
second_model_mathes_filtered['Prediction_points'] = second_model_mathes_filtered.apply(predictions_points, axis=1)

In [ ]:
def correctness_score(row):
    if row['TOI_result'] == row['Prediction_score']:
        val = 1
    elif row['TOI_result'] != row['Prediction_score']:
        val = 0
    return val
second_model_mathes_filtered['correct_score'] = second_model_mathes_filtered.apply(correctness_score, axis=1)

def correctness_points(row):
    if row['TOI_result'] == row['Prediction_points']:
        val = 1
    elif row['TOI_result'] != row['Prediction_points']:
        val = 0
    return val
second_model_mathes_filtered['correct_points'] = second_model_mathes_filtered.apply(correctness_points, axis=1)

In [ ]:
second_model_mathes_filteredd = second_model_mathes_filtered.iloc[1:].copy() #Remove the first row as we can not use model on it, so the resoult is wrong

In [ ]:
accuracy_score_model = round((sum(second_model_mathes_filteredd['correct_score'])/max((second_model_mathes_filteredd['matches_before_this_match']))*100),2)
if second_model_mathes_filteredd.shape[0] == 0:
    print('Can not determine accuracy of the modela as there was not enoug matches betwen',team_1, 'and', team_2)
else:
    print('Accuracy of model based on wins in previous duels is',accuracy_score_model,'%')

In [ ]:
accuracy_points_model = round((sum(second_model_mathes_filteredd['correct_points'])/max((second_model_mathes_filteredd['matches_before_this_match']))*100),2)
if second_model_mathes_filteredd.shape[0] == 0:
    print('Can not determine accuracy of the modela as there was not enoug matches betwen',team_1, 'and', team_2)
else:
    print('Accuracy of model based on points gained in previous duels is',accuracy_points_model,'%')

In [ ]:
second_model_mathes_filteredd